# 1. 오픈 API 활용 데이터 불러오기

In [1]:
# 라이브러리 불러오기
import pandas as pd
import requests
from pandas.io.json import json_normalize

In [2]:
# 데이터 접속 주소 입력
url = 'http://kosis.kr/openapi/statisticsData.do?method=getList&apiKey=NzliOTY3YjZkOTNmN2RkNzNhMjU4MmVjZDg1MDdhYjg=&format=json&jsonVD=Y&userStatsId=nano8990/101/DT_1G1B002/2/1/20190417152056&prdSe=M&newEstPrdCnt=1'

# 정상 여부 확인 (200 정상)
urlResponse = requests.get(url)
print(urlResponse)

# JSON 데이터 획득
jsonData = urlResponse.json()

# PandasDataframe 변환
df = json_normalize(jsonData)
# df.to_csv("a.csv", encoding = 'ms949')

<Response [200]>


,C1,C1_NM,C1_NM_ENG,C1_OBJ_NM,C1_OBJ_NM_ENG,C2,C2_NM,C2_NM_ENG,C2_OBJ_NM,C2_OBJ_NM_ENG,...,ITM_ID,ITM_NM,ITM_NM_ENG,ORG_ID,PRD_DE,PRD_SE,TBL_ID,TBL_NM,UNIT_NM,UNIT_NM_ENG
0,0,수주총액,Total amount of orders received,발주자별,By the kind of orderer,0,계,Total,공종별,By the kind of construction works,...,T10,수주액,Value of order received,101,201902,M,DT_1G1B002,발주자/공종별 건설수주액(경상),백만원,Million Won
1,0,수주총액,Total amount of orders received,발주자별,By the kind of orderer,1,건축,Building,공종별,By the kind of construction works,...,T10,수주액,Value of order received,101,201902,M,DT_1G1B002,발주자/공종별 건설수주액(경상),백만원,Million Won
2,0,수주총액,Total amount of orders received,발주자별,By the kind of orderer,2,토목,Civil engineering,공종별,By the kind of construction works,...,T10,수주액,Value of order received,101,201902,M,DT_1G1B002,발주자/공종별 건설수주액(경상),백만원,Million Won
3,1,공공부문,Public sector,발주자별,By the kind of orderer,0,계,Total,공종별,By the kind of construction works,...,T10,수주액,Value of order received,101,201902,M,DT_1G1B002,발주자/공종별 건설수주액(경상),백만원,Million Won
4,1,공공부문,Public sector,발주자별,By the kind of orderer,1,건축,Building,공종별,By the kind of construction works,...,T10,수주액,Value of order received,101,201902,M,DT_1G1B002,발주자/공종별 건설수주액(경상),백만원,Million Won


# 2. 공공데이터 포털 API 데이터 불러오기(XML) 

## Python활용 불러오기

In [29]:
# 라이브러리 선언
import urllib.request
import pandas as pd
from bs4 import BeautifulSoup
import requests

In [2]:
# 데이터 불러오기
xmlurl = 'http://openapi.molit.go.kr/OpenAPI_ToolInstallPackage/service/rest/RTMSOBJSvc/getRTMSDataSvcAptTradeDev?serviceKey=AT33s775KYpJOkUBJu0dxkJuUeIfDIOJRzAH084EQS3JN%2BzFjErLHuk%2FGZa9L4gBTSGCzeA69tI9PwLp7B37IQ%3D%3D&pageNo=1&startPage=1&numOfRows=10&pageSize=10&LAWD_CD=11110&DEAL_YMD=201512'
request = urllib.request.Request(xmlurl)
request.get_method = lambda: 'GET'
response_body = urllib.request.urlopen(request).read()
fullXmlString = str(response_body, "utf-8")

In [15]:
# 라이브러리 선언
import xml.etree.ElementTree as ET
from xml.etree.ElementTree import parse, Element

In [29]:
# 모든 데이터를 담을 리스트 선언
result = []
# xml 라이브러리 표준
root = ET.fromstring(fullXmlString)
# body -> items -> item  모두 선택
elements = root.findall('body/items/item')
# for 문을 돌면서 리스트에 모든 행 데이터 담기
for item in elements:
    try:
        item_list = []
        item_list.append(item.find('거래금액').text)
        item_list.append(item.find('건축년도').text)
        result.append(item_list)
    except Exception as e:
        print('This  row will be ignored', item_list)

In [6]:
# 디버깅 코드
elements[0].find('거래금액').text

'    82,500'

In [19]:
elements[8].find('*').text

'    44,000'

In [31]:
# 리스트 -> 데이터 프레임으로 변환
finalResult = pd.DataFrame(result)
finalResult.columns = ["거래금액", "건축년도"]
finalResult.head()

,거래금액,건축년도
0,"82,500",2008
1,"60,000",1981
2,"130,000",2004
3,"105,000",2004
4,"120,000",2003


In [40]:
# 데이터 접속 주소 입력
url = 'https://www.data.go.kr/dataset/fileDownload.do?atchFileId=FILE_000000001420537&fileDetailSn=1&publicDataDetailPk=uddi:1187ad78-3933-43e9-92ae-9ed15c7096d8_201712281357'

data = pd.read_csv(url, encoding = 'ms949')
data.head()

,연도,월,전국PIR,서울PIR,부산PIR,대구PIR,인천PIR,광주PIR,대전PIR,울산PIR,...,세종LIR,경기LIR,강원LIR,충북LIR,충남LIR,전북LIR,전남LIR,경북LIR,경남LIR,제주LIR
0,2004,3,4.21,4.89,3.95,3.73,4.65,2.81,4.68,2.66,...,-,2.42,1.44,1.53,1.21,1.90,1.42,1.31,2.06,1.28
1,2004,4,4.39,5.59,3.91,3.88,4.59,2.92,3.83,2.74,...,-,2.79,1.18,1.71,1.97,1.57,1.33,1.26,1.92,2.46
2,2004,5,4.19,5.14,4.90,3.83,4.78,3.41,4.19,2.93,...,-,2.22,1.44,1.72,1.85,1.75,1.31,1.50,1.85,2.14
3,2004,6,4.09,4.38,4.20,3.77,4.30,2.83,4.19,2.81,...,-,2.62,1.53,1.57,1.79,1.64,1.19,1.56,1.89,1.81
4,2004,7,4.29,5.88,4.43,4.28,4.81,3.32,4.35,2.54,...,-,2.41,1.68,1.91,2.15,1.64,1.36,1.50,1.93,2.63


In [ ]:
def xmlProcess(url):
    response = requests.get(url)
    # Check if page is up
    if response.status_code == 200:
        # Convert webpage to %Data
        Data = BeautifulSoup(response.text, 'lxml-xml')
        result = []
        rows = 0
        columnName = []
        # search Item all item tag
        iterData = Data.find_all('item')
        for item in iterData:
            item_list = []
            # Fill the value in one row
            for tag in item.find_all():
                try:
                    tagname = tag.name
                    if rows == 0:
                        columnName.append(tagname)
                    item_list.append(item.find(tagname).text)
                except Exception as e:
                    print("This row will be ignored. ", item_list)
            rows = rows + 1
            result.append(item_list)
    finalResult = pd.DataFrame(result)
    finalResult.columns = columnName
    print(finalResult)

In [13]:
xmlurl = 'http://apis.data.go.kr/1611000/nsdi/BuildingUseService/wfs/getBuildingUseWFS?ServiceKey=aDTqxJKNQ6XA6akRhrCF1ZSGXu8uk6HwyjXEWNetiKZEo%2FAD1M38g97KxPIXwL5k4RBnx%2BJwa6DzpTgn0G6AEg%3D%3D&maxfeatures=10&typename=F253&bbox=217694,448235,218608,449094,EPSG:5174&pnu=414501080010325&maxFeatures=10&resultType=results&srsName=EPSG:5174'

In [46]:
response = requests.get(xmlurl)
Data = BeautifulSoup(response.text, 'lxml-xml')

In [47]:
Data

<?xml version="1.0" encoding="utf-8"?>
<wfs:FeatureCollection numberOfFeatures="10" timeStamp="2019-04-24T02:25:31.179Z" xmlns:NSDI="http://openapi.nsdi.go.kr" xmlns:gml="http://www.opengis.net/gml" xmlns:wfs="http://www.opengis.net/wfs" xmlns:xs="http://www.w3.org/2001/XMLSchema" xmlns:xsi="http://www.w3.org/2001/XMLSchema-instance" xsi:schemaLocation="http://openapi.nsdi.go.kr http://10.20.10.46:8080/geoserver/NSDI/wfs?service=WFS&amp;version=1.1.0&amp;request=DescribeFeatureType&amp;typeName=NSDI%3AF253 http://www.opengis.net/wfs http://schemas.opengis.net/wfs/1.1.0/wfs.xsd"><gml:featureMember><NSDI:F253 gml:id="F253.18257198"><NSDI:SHAPE><gml:Polygon srsDimension="2" srsName="http://www.opengis.net/gml/srs/epsg.xml#5174"><gml:exterior><gml:LinearRing><gml:posList>217853.568 448978.337 217841.468 448975.857 217842.798 448971.337 217852.098 448972.997 217853.938 448975.767 217853.568 448978.337</gml:posList></gml:LinearRing></gml:exterior></gml:Polygon></NSDI:SHAPE><NSDI:GIS_IDNTFC_N

In [74]:
result = []
rows = 0
columnName = []
# search Item all item tag
iterData = Data.find_all('GIS_IDNTFC_NO')
iterData

[<NSDI:GIS_IDNTFC_NO>0000217847824489747900000000</NSDI:GIS_IDNTFC_NO>,
 <NSDI:GIS_IDNTFC_NO>1983217822794489688700000000</NSDI:GIS_IDNTFC_NO>,
 <NSDI:GIS_IDNTFC_NO>2002217869724489764000000000</NSDI:GIS_IDNTFC_NO>,
 <NSDI:GIS_IDNTFC_NO>1996217862494489661600000000</NSDI:GIS_IDNTFC_NO>,
 <NSDI:GIS_IDNTFC_NO>1983217859254489583100000000</NSDI:GIS_IDNTFC_NO>,
 <NSDI:GIS_IDNTFC_NO>1985217869464489562200000000</NSDI:GIS_IDNTFC_NO>,
 <NSDI:GIS_IDNTFC_NO>1983217875134489630400000000</NSDI:GIS_IDNTFC_NO>,
 <NSDI:GIS_IDNTFC_NO>1983217897854489753800000000</NSDI:GIS_IDNTFC_NO>,
 <NSDI:GIS_IDNTFC_NO>1983217892294489665200000000</NSDI:GIS_IDNTFC_NO>,
 <NSDI:GIS_IDNTFC_NO>1983217886274489560400000000</NSDI:GIS_IDNTFC_NO>]

In [96]:
item = iterData[0]
item_list = []
for tag in item.find_all():
    try:
        tagname = tag.name
        if rows == 0:
            columnName.append(tagname)
        item_list.append(item.find(tagname).text)
    except Exception as e:
        print("This row will be ignored. ", item_list)
rows = rows + 1
result.append(item_list)

In [99]:
item.find_all()

[]

In [91]:
iterData = Data.find_all('PNU')
for item in iterData:
    item_list = []
    # Fill the value in one row
    for tag in item.find_all():
        try:
            tagname = tag.name
            if rows == 0:
                columnName.append(tagname)
            item_list.append(item.find(tagname).text)
        except Exception as e:
            print("This row will be ignored. ", item_list)
    rows = rows + 1
    result.append(item_list)

In [92]:
item_list

[]

In [87]:
# 모든 데이터를 담을 리스트 선언
result = []
# xml 라이브러리 표준
root = ET.fromstring(fullXmlString)
# body -> items -> item  모두 선택
elements = root.findall('<>')

In [24]:
elements

[]

In [ ]:
# for 문을 돌면서 리스트에 모든 행 데이터 담기
for item in elements:
    try:
        item_list = []
        item_list.append(item.find('거래금액').text)
        item_list.append(item.find('건축년도').text)
        result.append(item_list)
    except Exception as e:
        print('This  row will be ignored', item_list)